Курсовой проект "Создание чат-бота в Telegram" 
---

## Ноутбук № 3: рабочий код бота



Устанавливаем библиотеку python-telegram-bot. Ссылка на документацию https://github.com/python-telegram-bot/python-telegram-bot

In [1]:
import pandas as pd
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
import logging
import pickle
import os
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import linear_model

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

In [2]:
path = '../../../data/spam_detection/'

## Подготовка модели

In [3]:
exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in exclude]
    return " ".join(txt)

In [4]:
df = pd.read_pickle(os.path.join(path, 'df_processed.pkl'))
print(df.shape)
df.head(3)

(18393, 2)


,msg,spam
0,че за возня опять возле верный,0
1,что то похожий на стрельба быть и сейчас три м...,0
2,а ктоть в курс что за стрельба,0


In [5]:
pkl_path_filename = os.path.join(path, 'lr_model.pkl')
with open(pkl_path_filename, 'rb') as file:
    lr = pickle.load(file)
lr

LogisticRegression(class_weight='balanced')

In [6]:
pkl_path_filename = os.path.join(path, 'count_vectorizer.pkl')
with open(pkl_path_filename, 'rb') as file:
    count_vect = pickle.load(file)
count_vect

HashingVectorizer(n_features=4800)

In [7]:
def check_spam(text):
    text = preprocess_text(text)
    return lr.predict(count_vect.transform(pd.Series(text)))

## Телеграм бот

In [8]:
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO
                    )

logger = logging.getLogger()

In [9]:
def start(update: Update, context: CallbackContext):
    update.message.reply_text('Привет!')

def echo(update: Update, context: CallbackContext):
    msg = update.message.text
    update.message.reply_text('Ваше сообщение: ' + msg)
    
    if check_spam(msg) == 1:
        update.message.reply_text('Это спам!')
    else:
        update.message.reply_text('Это не спам')

In [10]:
with open(os.path.join(path, 'token.txt'), "r") as file:
    token = file.read()

In [11]:
updater = Updater(token=token, use_context=True) 
dispatcher = updater.dispatcher

In [12]:
dispatcher.add_handler(CommandHandler("start", start))

dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

In [13]:
updater.start_polling(drop_pending_updates=True)
updater.idle()

2021-08-29 17:00:28,766 - apscheduler.scheduler - INFO - Scheduler started
2021-08-29 17:01:18,308 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2021-08-29 17:01:18,308 - apscheduler.scheduler - INFO - Scheduler has been shut down


telegram.ext подмодуль, предоставлющий простой в использовании интерфейс. Он состоит из нескольких классов, но двумя наиболее важными из них являются telegram.ext.Updater и telegram.ext.Dispatcher.
Класс Updater непрерывно получает новые обновления  из телеграмма и передает их в Dispatcher класс.
Если вы создадите Updater объект, он создаст Dispatcher для вас и свяжет их вместе.


Затем вы можете зарегистрировать обработчики разных типов в Dispatcher, который будет сортировать обновления, извлеченные в Updater соответствии с зарегистрированными вами обработчиками, и доставлять их в функцию обратного вызова, которую вы определили.


Напишем 2 обработчика команд. Это callback-функции, которые будут вызываться тогда, когда будет получено обновление. Напишем две таких функции для команды /start и для обычного любого текстового сообщения. В качестве аргументов туда передаются два параметра: bot и update. Bot содержит необходимые методы для взаимодействия с API, а update содержит данные о пришедшем сообщении.


Каждый обработчик является экземпляром любого подкласса класса telegram.ext.Handler . Библиотека предоставляет классы-обработчики почти для всех вариантов использования, но если вам нужно что-то очень конкретное, вы также можете создать подкласс Handler самостоятельно.

Теперь осталось лишь присвоить уведомлениям эти обработчики и начать поиск обновлений.
Цель состоит в том, чтобы вызывать эту функцию каждый раз, когда бот получает сообщение Telegram, содержащее /start команду. Для этого можно использовать CommandHandler(один из предоставленных подклассов Handler) и зарегистрировать его в диспетчере:
Класс Filters содержит ряд функций, которые фильтруют входящие сообщения на наличие текста, изображений, обновлений статуса и т.д. 
Любое сообщение, которое возвращает True в хотя бы одном из переданных фильтров, MessageHandler примет. Также можно написать свои собственные фильтры.


Создаём папку Bot, в которой потом создаём файл bot.py. Вы можете создать просто текстовый блокнот и вместо расширения .txt напишите .py Собираем код нашего бота. Открываем консоль и переходим в директорию с файлом, и запускаем python3 bot.py Бот будет работать пока будет открыто окно консоли.

In [6]:
# def startCommand(bot, update):
#     bot.send_message(chat_id=update.message.chat_id, text='Добрый день')
    
# def textMessage(bot, update):
#     response = 'Ваше сообщение принял ' + update.message.text # формируем текст ответа
#     bot.send_message(chat_id=update.message.chat_id, text=response)

In [ ]:
# dispatcher.add_handler(start_command_handler)
# dispatcher.add_handler(text_message_handler)

In [ ]:
# start_command_handler = CommandHandler('start', startCommand)
# text_message_handler = MessageHandler(Filters.text, textMessage)
# dispatcher.add_handler(start_command_handler)
# dispatcher.add_handler(text_message_handler)
# updater.start_polling(clean=True)
# # Останавливаем бота, если были нажаты Ctrl + C
# updater.idle()